In [8]:
import pandas as pd 
import matplotlib.pylab as plt
import numpy as np
from pathlib import Path
import os

In [9]:
generic_endpoints = pd.read_excel('/mnt/ncshare/ozkilim/BRCA/data/TCGA_metadata/TCGA-CDR-SupplementalTableS1.xlsx',sheet_name="ExtraEndpoints") # Uncomment this if you're reading from a CSV
generic_endpoints.head()

,Unnamed: 0,bcr_patient_barcode,type,PFI.1,PFI.time.1,PFI.2,PFI.time.2,PFS,PFS.time,DSS_cr,DSS.time.cr,DFI.cr,DFI.time.cr,PFI.cr,PFI.time.cr,PFI.1.cr,PFI.time.1.cr,PFI.2.cr,PFI.time.2.cr
0,1,TCGA-OR-A5J1,ACC,1.0,754.0,1.0,754.0,1.0,754.0,1.0,1355.0,1.0,754.0,1.0,754.0,1.0,754.0,1.0,754.0
1,2,TCGA-OR-A5J2,ACC,1.0,289.0,1.0,289.0,1.0,289.0,1.0,1677.0,NaN,NaN,1.0,289.0,1.0,289.0,1.0,289.0
2,3,TCGA-OR-A5J3,ACC,1.0,53.0,1.0,53.0,1.0,53.0,0.0,2091.0,1.0,53.0,1.0,53.0,1.0,53.0,1.0,53.0
3,4,TCGA-OR-A5J4,ACC,1.0,126.0,1.0,126.0,1.0,126.0,1.0,423.0,NaN,NaN,1.0,126.0,1.0,126.0,1.0,126.0
4,5,TCGA-OR-A5J5,ACC,1.0,50.0,1.0,50.0,1.0,50.0,1.0,365.0,NaN,NaN,1.0,50.0,1.0,50.0,1.0,50.0


In [10]:
import pandas as pd
import os

# Path to the folder containing the files
folder_path = '/tank/WSI_data/TCGA_WSI_organized/TCGA-GBM/CLAM/level_0/ViT/h5_files/'

# List all filenames in the folder
filenames = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]


# Extract the relevant part of the filename (adjust this based on your filename structure)
# Assuming the bcr_patient_barcode is at the start of the filename, followed by a delimiter like '-'
extracted_identifiers = ['-'.join(filename.split('.')[0].split('-')[:3]) for filename in filenames]

slide_ids = [f[:-3] for f in filenames]

# Create a new DataFrame from filenames
file_df = pd.DataFrame({'slide_id': slide_ids, 'bcr_patient_barcode': extracted_identifiers})

# Merge the new DataFrame with the existing one
# This ensures that all filenames are included and matched data from the original DataFrame is added
merged_df = file_df.merge(generic_endpoints, on='bcr_patient_barcode', how='left')

In [11]:
merged_df.head(50)

# Binarize but have onother set. 

median_pfs = merged_df['PFS.time'].median()
print(median_pfs)
# Apply the median as a threshold to create a new binary column
# Patients with PFS less than the median are classified as 1, and those with PFS greater than or equal to the median as 0
merged_df['PFS_binary'] = (merged_df['PFS.time'] < median_pfs).astype(int) # 6 month PFS? ... what is clincally viable?...  

merged_df.rename(columns={'bcr_patient_barcode': 'case_id'}, inplace=True)

COAD_task = merged_df[['case_id', 'slide_id', 'PFS.time','PFS_binary']]

189.0


In [12]:
COAD_task['slide_type'] = COAD_task['slide_id'].str.split('-').str[5].apply(lambda x: x.split('.')[0])
COAD_task

/home/ozkilim/anaconda3/envs/clam/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,case_id,slide_id,PFS.time,PFS_binary,slide_type
0,TCGA-06-0145,TCGA-06-0145-01A-01-TS1.6474f743-1f0a-4eef-a21...,71.0,1,TS1
1,TCGA-12-0772,TCGA-12-0772-01Z-00-DX3.c8aa40ec-3007-457e-a35...,607.0,0,DX3
2,TCGA-14-1453,TCGA-14-1453-01Z-00-DX1.214c0e86-4eaa-495e-ab5...,35.0,1,DX1
3,TCGA-06-0146,TCGA-06-0146-01A-01-TS1.0fd0511f-cbe2-4ec0-abb...,530.0,0,TS1
4,TCGA-08-0386,TCGA-08-0386-01A-01-BS1.066ec05b-4bd8-4727-89a...,427.0,0,BS1
...,...,...,...,...,...
2036,TCGA-06-0644,TCGA-06-0644-01Z-00-DX1.bbce7e5c-6df3-4963-ac7...,85.0,1,DX1
2037,TCGA-12-1094,TCGA-12-1094-01Z-00-DX2.9B90CBA4-A9CB-48B4-A59...,186.0,1,DX2
2038,TCGA-14-0867,TCGA-14-0867-01Z-00-DX2.f2d7d9bd-85d4-47f0-a14...,62.0,1,DX2
2039,TCGA-08-0520,TCGA-08-0520-01A-01-BS1.f1703e2a-8221-42d8-914...,105.0,1,BS1


In [13]:
# COAD_task['slide_type'].value_counts()

# COAD_task_FFPE = COAD_task[COAD_task['slide_type'] == 'DX1']

COAD_task.to_csv("/mnt/ncshare/ozkilim/BRCA/data/tasks/TCGA_GBM_PFS_All.csv",index=None)

In [14]:
# COAD_task_FFPE["PFS_binary"].value_counts()